# sprint14 論文読解

## Faster R-CNN: Towards Real-Time Object Detection with Region Proposal Networks

## （１）物体検出の分野にはどういった手法が存在したか

　物体検出（object detection）は、領域提案（object proposal methods)からの入力を受け入れる外部モジュールとなっている。  
　物体検出の分野には、  
・R-CNN  
・fast R-CNN  
・Adaptively sized pooling(SPP)  
・semantic segmentation  
　がある。  
 
　なお、overfeat methodsやmultibox methodsは、領域提案の分野の手法であると解釈される。  

## （２）Fasterとあるが、どういった仕組みで高速化したのか。

　まずfast R-CNNでは、R-CNNで生成される特徴マップに対してselective searchのアルゴリズムなどを適用した領域提案をすることで、R-CNNが高速化された。領域提案された箇所は、ROIプーリングにより画像データが揃えられる。    

　Faseter-R-CNNでは、さらにRPN（region proposal network）を導入して、selective searchによる領域提案と特徴マップ生成の処理を共通化させることにより高速化される。  
　論文の図２のように、前段となるRPNの領域提案と、後段となるFast R-CNNによる物体検知とで共通化された畳み込み層が設けられる（RPN、Fast R-CNNの学習については後述する）。  
 　
　RFNでは、特徴マップ生成のフィルタを走らせる際に、anchoringを行っている（特徴マップの各ピクセル毎に、９つのアンカーを適用して、各アンカーに対してのground truth box との重複度合いを判断する）。  

## (3) One-Stageの手法とTwo-Stageの手法はどう違うのか。

　1stage : overfeat による領域提案＋fast R-CNNによる物体検出  
　2stage : Faster R-CNN(RPNによる領域提案＋畳み込み層が共通化されたfast R-CNNによる物体検出。）  
　
　1stageの手法では、overfeatでの領域提案に畳み込み層が存在せず、領域提案→物体検出が一方通行の学習・予測となっている。  
　2stageの手法では下記のような4段階の学習となっていると思われる。  
１．RPN単体での学習。（アンカーとground truthとの差分に対応したコスト関数を使用した、共通畳み込み層とＲＰＮユニーク層の学習）  
２．RPNからの領域提案（アンカー）を入力とした、共通畳み込み層とFast R-CNNユニーク層の学習。  
３．共通畳み込み層のパラメータを固定して、RPNユニーク層のfine tuning（学習）  
４．共通畳み込み層のパラメータを固定して、Fast R-CNNユニーク層のfine tuning（学習）
（１～４の繰り返し）  

　また、1stageの手法では、オブジェクトのラベルが１段階での検出であるのに対し、2stageの手法では、オブジェクトのラベルが２段階の検出（class agnostic proposalと、class specific detection）となっている。  

ＭＥＭＯ：  
　なお予測の際は、インプット→領域提案→「ＲＰＮからの出力（領域提案）」と「共通畳み込み層からの出力（特徴マップ）」を、Ｒ－ＣＮＮへの入力として、Ｒ－ＣＮＮ内でネットワークが２つに分岐してラベルとbounding box を出力するものと思われる。  
 　ＲＰＮからの領域提案は複数候補であり、Ｒ－ＣＮＮからのbounding boxの出力は微調整・洗練されたものとなっていると思われる。  

## (4) RPNとは何か。  

  RPN(Region Proposal Network)。  
  
　RPNは、入力画像が畳み込まれて生成されるfeature mapを使って、効率化された領域提案を実現する畳み込みネットワークである。  
サイズやアスペクト比が固定された矩形領域のアンカーがハイパーパラメータとして設定されており、いずれかのアンカーとそのobject scoreを出力するモジュールとなっている（背景or物体の判断を行う）。

## (5) RoIプーリングとは何か。

　ＲＰＮから出力される領域提案（アンカーの領域）に対応する特徴マップに(max) pooling 処理を施して、7×7や3×3といったsame spatial sizeに丸め込む。  
 これにより多様なサイズ（具体的には９種類）でＲＦＮから出力される領域提案を、後段のFast R-CNNに入力する際の次元を揃えることができる。

## (6) Anchorのサイズはどうするのが適切か。

　アンカーは、特徴マップ上でウィンドウをスライドさせる毎に設定される（ウィンドウの中央で位置決め）。  
　アンカーとしては、大きさや縦横比が異なる９種類のものが予めされており、以下の基準にて各スライドポジションごとに決定される。  
・ground truth box との重複についての指標である、IOU指標が最大となるanchor。  
・あるいは、IOU指標がいずれかのground truth box と0.7以上となるanchor。  

　アンカーのアスペクト比やサイズは、ある程度の種類を用意しておけばよく、あまり注意を要するハイパーパラメータではない。  

## (7) 何というデータセットを使い、先行研究に比べどういった指標値が得られているか。

データセット：pascal VOC 2007、2012、MS COCO  
評価指標はｍＡＰ  
図１１によると、先行研究（fast R CNN）と比べてｍＡＰが３％程度向上した。